## Data Pre-Processing

In [12]:
# Import modules
import matplotlib.pyplot as plt
import pandas as pd

# Import dataset
dataset = pd.read_csv('datasets/multiple-linear-regression-data.csv')

# Separate dataset (X) and dependent vector (y)
X = dataset.drop(columns=['Profit'])
y = dataset.filter(items=['Profit'])

# Encode Categorical Data
dummy_vars = ['State']
X_categorical = pd.get_dummies(X[dummy_vars])
X = pd.concat([X_categorical, X.drop(columns=dummy_vars)], axis = 1, sort=False)

# Avoiding the dummy variable trap
X.drop(columns=['State_California'], inplace=True)

# Splitting Training and Test Set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Multiple Linear Regression

In [14]:
# Fitting Simple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predicting the Test set results
y_pred = pd.DataFrame(lr.predict(X_test), columns=y_train.columns.values)

## Comparing Predictions

In [17]:
for i in range(len(y_pred['Profit'])):
    error = y_test['Profit'].values[i] - y_pred['Profit'].values[i]

error:  267.17840203578817
error:  11677.122391842597
error:  13674.211548250518
error:  5822.7314874140575
error:  12512.907789456425
error:  -11152.932301633817
error:  13377.36790324049
error:  -1308.1737468803622
error:  -3617.185330116641
error:  -1733.1256955016288
